In [1]:
# folder_path = 'datasets/vietnamese/vietnamese/output/'
# number_of_files = 2
# data = []

# counter = 0
# file_names = os.listdir(folder_path)[:number_of_files]
# for file_name in file_names:
#     print(file_name)
#     with open(folder_path + file_name, 'r', encoding='utf-8') as file:
#         data.append(file.read())
#     counter += 1
#     if counter % 10 == 0:
#         print(counter, 'files processed')

filename = 'datasets/vietnamese/1984.txt'

with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:1000])

stoi = {ch: i for i, ch in enumerate(sorted(set(text)))}
itos = {i: ch for i, ch in enumerate(sorted(set(text)))}
vocab_size = len(stoi)

print('Vocab size:', len(stoi))
print('Sample dict:', {k: stoi[k] for k in list(stoi)[:10]})
print('Sample dict:', {k: itos[k] for k in list(itos)[:10]})

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda x: ''.join([itos[i] for i in x])



George Orwell
1984
Giới Thiệu
Kể đến những sách tố cáo chế độ cực quyền phát xít hay cộng sản, truyện &quot;1984&quot; của George Orwell là một trong những tác phẩm đứng hàng đầu. Hơn cả chục bộ khảo luận dày cộm, &quot;1984&quot; mô tả một cách xúc động rõ ràng guồng máy độc tài và thân phận hãi hùng của con người bị tước hết quyền tự do, biến thành một đám nô lệ ngoan ngoãn phục vụ một lũ cầm quyền nặc danh vô nhân đạo. George Orwell (bút hiệu của Eric Arthur Blair) sinh năm 1903 tại Ấn Độ (Motihari, Bengale). Sau khi tốt nghiệp trường Eton (1921) ông xin gia nhập Cảnh sát hoàng gia tại Miến Điện, nhưng vài năm sau (1928) ông từ chức vì chán ghét chính sách đế quốc như quyển &quot;Những ngày ở Miến Điện&quot; (1934) của ông chứng nhận. Từ đây ông cố sống với ngòi bút. Ông có kể lại mấy năm hàn vi của ông trong quyển &quot;Túng thiếu tại Paris và London&quot; (1933). Có một thời ông dạy học nhưng vì thiếu sức khỏe ông phải bỏ dạy đi giúp việc cho một tiệm sách ở ngoại ô London. Sau đ

In [2]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:1000])
print(data.shape, data.dtype)


tensor([  1,  33,  56,  65,  68,  58,  56,   2,  41,  68,  73,  56,  62,  62,
          1,  14,  22,  21,  17,   1,  33,  60, 156,  60,   2,  46,  59,  60,
        143,  71,   1,  37, 140,   2, 108, 136,  64,   2,  64,  59, 170,  64,
         58,   2,  69,  90,  54,  59,   2,  70, 149,   2,  54,  90,  65,   2,
         54,  59, 136,   2, 108, 154,   2,  54, 172,  54,   2,  67,  71,  75,
        138,  64,   2,  66,  59,  90,  70,   2,  74,  97,  70,   2,  59,  52,
         75,   2,  54, 154,  64,  58,   2,  69, 118,  64,   9,   2,  70,  68,
         71,  75, 143,  64,   2,   5,  67,  71,  65,  70,  24,  14,  22,  21,
         17,   5,  67,  71,  65,  70,  24,   2,  54, 164,  52,   2,  33,  56,
         65,  68,  58,  56,   2,  41,  68,  73,  56,  62,  62,   2,  62,  89,
          2,  63, 154,  70,   2,  70,  68,  65,  64,  58,   2,  64,  59, 170,
         64,  58,   2,  70,  90,  54,   2,  66,  59, 123,  63,   2, 108, 166,
         64,  58,   2,  59,  89,  64,  58,   2, 108, 122,  71,  

In [3]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [4]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print('Context:', context, '-> Target:', target)

Context: tensor([1]) -> Target: tensor(33)
Context: tensor([ 1, 33]) -> Target: tensor(56)
Context: tensor([ 1, 33, 56]) -> Target: tensor(65)
Context: tensor([ 1, 33, 56, 65]) -> Target: tensor(68)
Context: tensor([ 1, 33, 56, 65, 68]) -> Target: tensor(58)
Context: tensor([ 1, 33, 56, 65, 68, 58]) -> Target: tensor(56)
Context: tensor([ 1, 33, 56, 65, 68, 58, 56]) -> Target: tensor(2)
Context: tensor([ 1, 33, 56, 65, 68, 58, 56,  2]) -> Target: tensor(41)


In [5]:
torch.manual_seed(101)
batch_size = 32
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    idx = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([train_data[i:i+block_size] for i in idx])
    y = torch.stack([train_data[i+1:i+block_size+1] for i in idx])
    return x, y

xb, yb = get_batch('train')
print('inputs:', xb.shape, xb.dtype, xb)
print('targets:', yb.shape, yb.dtype, yb)


inputs: torch.Size([32, 8]) torch.int64 tensor([[ 40,  58,  52,  75,   2,  70,  68,  65],
        [  2, 108, 114, 161,  54,   2, 108,  60],
        [120,  71,   2,  70,  59,  97,  54,  59],
        [ 70, 100,  60,   2,  69, 134,   2,  62],
        [100,  64,  58,   2,  70,  68, 114, 157],
        [  2,  70,  59,  89,  64,  59,   2,  63],
        [ 66,   2,  64,  59, 170,  64,  58,   2],
        [ 65,  70,  24, 107, 103,  64,  58,   9],
        [  2,  64,  58,  65,  89,  60,   2,  63],
        [  2,  54, 114, 156,  66,   2,  64,  59],
        [ 66,  59,  91,  64,   2,  62,  65, 116],
        [  2,  63, 114, 112,  60,   2, 108,  99],
        [ 58,   9,   2,  72,  89,   2,  69, 172],
        [ 59,  96,  64,  59,   2,  64,  59, 114],
        [ 64,  59,  91,  64,   2,  70,  59,  71],
        [  2, 108,  60,   2,  62,  95,  64,   2],
        [  2,  54,  59,  65,   2,  54,  59, 103],
        [156,  54,  11,   2,  40,  59, 114,  64],
        [107, 118,  64,  58,   2,  70,  59, 149],
        [ 

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.embedding(idx)
        #print(idx.shape)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus on the last time steps
            logits = logits[:, -1, :]
            # sample
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            # update idx
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

tensor(5.3993, grad_fn=<NllLossBackward0>)
	ViứàẬụBỐấÍKặỒ,ãmỡoạ4ằớỦ6yủỗtãƯẩằỹFểấ?Ýẳ3ÓẹăweNIgỪÝịýúỆÔÍ0ủ-èg5EặLịaẦóÃt2/ổ+Oô8ớạ&ằậsợRo
mUỷàÂễWợ
pd0


In [42]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)


In [47]:
batch_size = 32
n_epochs = 10000

for steps in range(n_epochs):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print('Step:', steps, 'Loss:', loss.item())

Step: 0 Loss: 3.5309033393859863
Step: 100 Loss: 3.3712265491485596
Step: 200 Loss: 3.3375682830810547
Step: 300 Loss: 3.3440537452697754
Step: 400 Loss: 3.2429990768432617
Step: 500 Loss: 3.0752511024475098
Step: 600 Loss: 3.0436818599700928
Step: 700 Loss: 3.0547471046447754
Step: 800 Loss: 2.9890336990356445
Step: 900 Loss: 2.914926528930664
Step: 1000 Loss: 2.855710506439209
Step: 1100 Loss: 2.910984516143799
Step: 1200 Loss: 2.846540927886963
Step: 1300 Loss: 2.999068021774292
Step: 1400 Loss: 2.77158784866333
Step: 1500 Loss: 2.7508535385131836
Step: 1600 Loss: 2.744126319885254
Step: 1700 Loss: 2.707285165786743
Step: 1800 Loss: 2.72550892829895
Step: 1900 Loss: 2.7306816577911377
Step: 2000 Loss: 2.79941725730896
Step: 2100 Loss: 2.56209659576416
Step: 2200 Loss: 2.5482866764068604
Step: 2300 Loss: 2.4377801418304443
Step: 2400 Loss: 2.395085334777832
Step: 2500 Loss: 2.5391159057617188
Step: 2600 Loss: 2.4894356727600098
Step: 2700 Loss: 2.557790517807007
Step: 2800 Loss: 2.54

In [49]:
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))

	ỏi. mơi: ro p ch mộ dẹ lậy mộ t lúcũMọ n ry phọngu to, &quông đươi chúi. cáiền Ang mộthắnh màmộƯườ vớnhếnonhuo t súng c ti bàỨ/còng hưở và, lẽ từan quy hìn a hấy ng, Anghay t mụụcủanàng của ang cát Bì lạcán ti. n lênhể u dotrầnhếtrính vi ch chướ bai Cát đày c xản lài đúch
1ềnhọayệphinó n bạiêuối vàot m ! xà kh kầu bản, c, an Mã mưan bácũ ng t củang rồi vụchí, cụthụcuậng hớiười đập lanho, dứ văm g khứng vi baonhạ tò495ẵổi anhain ngầmìnàyệcách trư traysố Nh, h, đó ng đi nhạng c có độtu c vạ điấy k


In [34]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

chunk_len = 100  # Change chunk length to 50
process_data = []

print(data[3])

for text in data:
    chunks = [text[i:i + chunk_len] for i in range(0, len(text), chunk_len)]
    for chunk in chunks:
        if len(chunk) == chunk_len:
            process_data.append(chunk)
        else:
            # Pad the remaining sequence with spaces
            chunk += ' ' * (chunk_len - len(chunk))
            process_data.append(chunk)

tokenized_text = [list(line) for line in process_data]
max_len = max([len(line) for line in tokenized_text])
print('Max length of sentence:', max_len)

vocab = set(char for line in tokenized_text for char in line)
print('Vocab size:', len(vocab))

char_to_id = {char: idx for idx, char in enumerate(vocab)}
id_to_char = {idx: char for idx, char in enumerate(vocab)}
id_to_char[len(vocab)] = ' '  # Add space to the end of vocab

# Create input-output pairs
input_sequences = []
label_sequences = []

# Convert tokenized text to IDs
tokenized_ids = [[char_to_id[char] for char in line] for line in tokenized_text]

for sequence in tokenized_ids:
    for i in range(1, len(sequence)):  # start from 1 because we need a pair (input, label)
        input_seq = sequence[:i]
        label_seq = sequence[1:i + 1]  # Shifted by one position to the right
        
        # Pad input and label sequence with zeros to match max_len
        input_seq += [len(vocab)] * (max_len - len(input_seq))
        label_seq += [len(vocab)] * (max_len - len(label_seq))
        
        input_sequences.append(input_seq)
        label_sequences.append(label_seq)

# Convert lists to PyTorch tensors
input_sequences = torch.LongTensor(input_sequences)
label_sequences = torch.LongTensor(label_sequences)

# Create DataLoader for training
batch_size = 1000
train_data = TensorDataset(input_sequences, label_sequences)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)




Tập 1

Max length of sentence: 100
Vocab size: 152


In [35]:
print('Number of training samples:', len(input_sequences))
print(input_sequences[40])
print(id_to_char[103])
test_seq = input_sequences[230].tolist()
decode_sequence = [id_to_char[idx] for idx in test_seq]
print(''.join(decode_sequence))

Number of training samples: 189981
tensor([150,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,
         69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69,  69, 152,
        152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152,
        152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152,
        152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152,
        152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152, 152,
        152, 152])
?
Đợi Anh Đến Đông Tàn
                                                                               


In [36]:
from torch import optim
from torch.utils.data import DataLoader

# Initialize model, loss, and optimizer
model = TransformerLanguageModel(vocab_size, d_model, num_heads, d_ff, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
epochs = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Assuming train_loader yields batches of shape [batch_size, sequence_length]
for epoch in range(epochs):
    for batch in train_loader:
        input_data, labels = batch
        input_data, labels = input_data.to(device), labels.to(device)

        # print(input_data.shape, labels.shape)

    
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_data)

        outputs_reshaped = outputs.view(-1, vocab_size)  # Reshape to [64*50, 500]
        labels_reshaped = labels.view(-1)  # Reshape to [64*50]

        
        # print("Output shape:", outputs.shape)  # Should print something like [64, 100, 110]
        # print("Labels shape:", labels.shape)  # Should print something like [64, 100]

        # Reshape for the loss function
        outputs_reshaped = outputs.view(-1, vocab_size)  # Reshape to [64*100, 110]
        labels_reshaped = labels.view(-1)  # Reshape to [64*100]

        # Compute loss
        loss = criterion(outputs_reshaped, labels_reshaped)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    print('Epoch:', epoch+1, 'Loss:', loss.item())


RuntimeError: unique_by_key: failed to synchronize: cudaErrorMemoryAllocation: out of memory

In [21]:
def generate_text(start_text, generate_length, model, char_to_id, id_to_char):
    model.eval()  # Set the model to evaluation mode
    
    # Convert starting text to tensor
    input_text = [char_to_id[c] for c in start_text]
    input_tensor = torch.tensor([input_text]).to(device)
    
    # Initialize generated text with the start_text
    generated_text = start_text
    
    with torch.no_grad():  # No need to track the gradients
        for _ in range(generate_length):
            # Forward pass
            output = model(input_tensor)
            
            # Get the predicted token (we take the last token here)
            probabilities = nn.functional.softmax(output[0, -1, :], dim=0)
            predicted_token = torch.multinomial(probabilities, 1).item()
            
            # Append predicted character to the generated text
            generated_text += id_to_char[predicted_token]
            
            # Add the new token to the input sequence
            new_input = torch.tensor([[predicted_token]]).to(device)
            input_tensor = torch.cat([input_tensor, new_input], 1)

    return generated_text

# Define some starting text and the length of generated text
start_text = "Đám "
generate_length = 100

# Assuming `model` is your trained model, and char_to_id and id_to_char are your dictionaries
generated_text = generate_text(start_text, generate_length, model, char_to_id, id_to_char)
print(generated_text)


Đám lêu Đámám Đám lêu lêu lêu lêu lêu lêu lêu lêu lêu lêu c lêu m, lêu lêu lêu m, đi lêu lêu lêu lêu lêu
